# Data Engineer challenge

Hola! en el siguiente notebook presento mi resolución y mi visión respecto al challenge propuesto, tratando de adentrarme en detalle lo más posible en cada una de mis respuestas, esperando que mis ideas tengan sentido respecto al rol que están buscando y que coincidan con la visión que también ustedes puedan tener para considerarme como miembro de su equipo.

## Overview

En consideración a los dos enfoques propuestos (tiempo y memoria) para cada problema y que cada uno de ellos hace uso del mismo set de datos, decidí establecer ciertos patrones reutilizables para cada enfoque, ya que a pesar de que los resultados de cada problema son específicos a cada uno de ellos, la forma de acceso a los datos podría ser la misma dependiendo del enfoque, lo que me permitiría enfocarme en tener menor tiempo de desarrollo y mayor tiempo para explayar mis respuestas de forma que se logren traspasar mis ideas y mi criterio en este documento.

Por otro lado a nivel general, me enfoqué en utilizar de mayor manera posible funciones nativas de Python, salvo una excepción que facilitaba el tiempo de desarrollo en el caso de los emojis; además, traté de utilizar ciertas facilidades del código de alto nivel de Python que creo que son interesantes, reutilizándolas dependiendo del enfoque a tratar y a la vez tratando de mantener la legibilidad pero sin dejar la simplicidad. Dicho esto, comenzaré detallando cada uno de los enfoques para luego mostrar cada problema en detalle.

### Memory

En el caso del enfoque en memoria, tuve en consideración principalmente el volumen del conjunto de datos, creo que la diferencia clave entre los dos enfoques está dado por el hecho de volcar todo el set de datos a la memoria o bien ir leyendo de registro en registro desde el archivo. Con esto en consideración, en el caso de la memoria, me incliné a utilizar diccionarios que permitieran almacenar ids de elementos e ir manteniendo un conteo de éstos, de forma que una vez fuera leyendo los registros uno a uno, ir detectando estos ids y aumentar los contadores de cada uno o agregándolos a los diccionarios según corresponda.

Respecto al código, utilicé un for loop para ir leyendo registro a registro manteniendo el archivo abierto pero sin cargarlo en memoria mediante open(), luego de parsearlos como objetos json, pude aplicar distintas metodologías dependiendo del problema a resolver para cada registro con este mismo for loop. Para el ordenamiento requerido en cada uno de los problemas, utilicé la función sorted, la cual permite ordenar los diccionarios anteriormente mencionados en base a los contadores almacenados para cada id.

### Time

Para este enfoque, me incliné en utilizar funciones de Python nativas que están diseñadas para trabajar con volúmenes de datos en memoria, como mencioné en el punto anterior considero que la diferencia clave de los enfoques está en el volcado de datos en memoria, por lo que en este caso leí el archivo completo para cargarlo en una variable str, luego en consideración a que el archivo es un newline splitted json, utilicé la función splitlines para separarlo en registros y aplicar la función de Python map para parsear cada registro como objeto json y así poder aplicar la metodología correspondiente al problema a resolver, finalmente utilicé el tipo de variable Counter de la librería collections de Python para retornar lo requerido por cada problema utilizando la función most_common, la cual automáticamente genera el ranking dado un conjunto de datos en memoria.

Respecto a los resultados, me parece importante ser transparente respecto a ellos ya que me parecieron interesantes, al momento de hacer el benchmarking respecto a la versión enfocada en memoria, la metodología del enfoque en memoria es levemente más rápida que la enfocada en tiempo por lo que ciertamente hay posibles mejoras para esta versión, si tuviera que mencionar alguna, creo que se puede trabajar de manera más óptima la data en memoria utilizando alguna librería de procesamiento de datos que pueda manejar mejor un volumen como el utilizado para el challenge, la primera librería que pienso que podría hacer esta mejora es pandas, sin embargo como parte positiva de este resultado, se puede observar que la cantidad de llamadas a funciones en efecto es más óptima en la versión enfocada en tiempo comparada con la versión enfocada en memoria, lo que se puede apreciar en el detalle de cada uno de los problemas a continuación.

## Resolución

Aplicando las metodologías descritas en los puntos anteriores, los resultados en código para cada uno de los problemas propuestos con su respectivo detalle:

In [1]:
# Code setup

## Imports

import json

from typing import List, Tuple
from datetime import datetime, date  # date requerido en el primer problema
from collections import Counter  # Requerido en la metodología del enfoque en tiempo
from emoji import EMOJI_DATA  # Requerido en el segundo problema
from re import compile  # Requerido en el tercer problema


## File path

file_path = "/Users/cammil.guzman/Downloads/farmers-protest-tweets-2021-2-4.json"

### Q1

#### Memory

En la resolución de este problema, se requería poder obtener la fecha del tweet en base al timestamp de fecha de cada registro además del usuario creador del tweet, luego en línea con la metodología del enfoque de memoria, almacenar la fecha y la combinación usuario/fecha en dos respectivos diccionarios para así llevar la cuenta de las cantidades de ambos, finalmente el output se reduce a un ordenamiento de estos contadores.

Comenzando, se inicializan los diccionarios: uno para llevar registro de la cantidad de tweets por cada fecha y otro para registrar la cantidad de tweets por cada fecha y usuario. Luego, se puede proceder a leer el archivo mediante open() y procesar cada uno de los registros mediante un for loop, convirtiendo a un objeto json con json.loads() cada uno de ellos permitiendo obtener específicamente la información necesaria minimizando el uso de memoria, obteniendo la fecha desde el valor identificado por la llave 'date' de cada tweet mediante la función split() y la función date.fromisoformat(), la primera de ellas permite separar un string en base a uno o más caracteres, buscando en este caso el caracter "T" el cual denota la separación entre la fecha y la hora, obteniendo de esta manera un string con la fecha el cual puede ser transformado en un objeto date de Python mediante la segunda función. A continuación, se obtiene el usuario desde el valor identificado por la llave 'username' el cual es parte del valor identificado por la llave 'user', no hay tratamiento que hacer sobre el username más que almacenarlo en el contador respectivo, en consideración a la fecha obtenida en el paso anterior. Finalmente, se ordenan estos diccionarios usando sorted, permitiendo obtener las 10 fechas donde hay mas tweets y el usuario que más publicaciones tiene por cada uno de esos días.

In [2]:
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    #Keep track of tweet count by date
    tweetCountByDate = {}

    #Keep track of user tweet count by date
    tweetUsersByDate = {}

    #Read file, by line
    with open(file_path, "r") as f:
        for line in f:
            #Parse line, get username and date
            parsedLine = json.loads(line)
            tweetDate = date.fromisoformat(parsedLine['date'].split('T')[0]) #Didn't use strptime due to local Python version
            tweetUsername = parsedLine['user']['username']

            #Add values to trackers
            tweetCountByDate[tweetDate] = tweetCountByDate.get(tweetDate, 0) + 1
            tweetUsersByDate[tweetDate] = tweetUsersByDate.get(tweetDate, {})
            tweetUsersByDate[tweetDate][tweetUsername] = tweetUsersByDate[tweetDate].get(tweetUsername, 0) + 1

        #Sort dates by tweet count then for each date sort their respective users to get the top one
        output = []
        sortedDates = sorted(tweetCountByDate, key=lambda d: tweetCountByDate[d], reverse=True)[0:10]
        for dateId in sortedDates:
            mostTweetsUser = sorted(tweetUsersByDate[dateId], key=lambda username: tweetUsersByDate[dateId][username], reverse=True)[0]
            output.append((dateId, mostTweetsUser))

        return output
    
print(
    q1_memory(file_path)
)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


#### Time

En consideración a la metodología del enfoque en tiempo, se busca volcar toda la información a la memoria para procesarla después en lotes, por lo que se requieren estructuras de datos para almacenarla y procesarla en línea con las funciones a utilizar, específicamente la función most_common() del tipo de datos Counter la cual requiere ejecutarse sobre una lista, en consecuencia se utilizan una lista para almacenar las fechas y un diccionario cuyas llaves son cada fecha y cuyos valores son una lista de usuarios que hayan hecho publicaciones en esa fecha, resultando en un diccionario de listas de usuarios ordenados por fecha. Finalmente, se pueden volcar los datos hacia estas variables las cuales pueden ser transformadas en variables tipo Counter sobre las cuales se puede ejecutar la función most_common(), obteniendo los resultados esperados con código de más alto nivel sin dejar de ser nativo.

En primer lugar se lee el archivo utilizando open() y read(), volcando todo el contenido del archivo en una variable tipo str, luego, se inicializan las variables mencionadas, una lista de fechas junto a un diccionario de llave fechas y valor una lista de usuarios. A continuación, se utiliza la función splitlines() para separar este string en líneas y mediante la función map() se aplica la función json.loads() sobre cada una de estas lineas, convirtiéndolas en objetos json, resultando en un iterable que puede ser procesado mediante un for loop y que por cada elemento de este se puede obtener la información almacenada en las llaves indicadas en la metodología anterior de esta pregunta (['date'] y ['user']['username']), para así almacenar la fecha en la lista de fechas y el usuario en la lista respectiva del diccionario, según la fecha obtenida. Finalmente, se puede crear una variable Counter en base a la lista de fechas y aplicando most_common(10) sobre ella se obtienen las top 10 fechas donde hay mas tweets, de la misma forma, por cada lista del diccionario se puede crear una variable tipo Counter y sobre cada una de ellas aplicar most_common(1) obteniendo el usuario que más publicaciones tiene para cada fecha.

In [3]:
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    #Open the whole file
    with open(file_path, "r") as f:
        rawContent = f.read()

    #Keep track of dates
    dates = []

    #Keep a list for each date, holding the username for each tweet on that date
    usersByDate = {}

    #Split the file by newlines
    for tweet in map(json.loads, rawContent.splitlines()): #Using splitlines will allow to skip newlines which may be present on the tweet's content
        #Parse tweet, get username and date, add them to the trackers
        tweetDate = date.fromisoformat(tweet['date'].split('T')[0]) #Didn't use strptime due to local Python version
        dates.append(tweetDate)
        tweetUsername = tweet['user']['username']
        if tweetDate in usersByDate:
            usersByDate[tweetDate].append(tweetUsername)
        else:
            usersByDate[tweetDate] = [tweetUsername]

    #Get the dates with most tweets
    mostTweetedDates = Counter(dates).most_common(10)

    #For the top 10 dates with most tweet counts get the most active user
    output = []
    for tweetDate in mostTweetedDates:
        dateId = tweetDate[0]
        output.append((dateId, Counter(usersByDate[dateId]).most_common(1)[0][0]))

    return output

print(
    q1_memory(file_path)
)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


### Q2

#### Memory

Para este caso, se requería detectar y contabilizar los emojis presentes en cada una de las publicaciones lo cual se realizó mediante un diccionario cuyas llaves son los emojis y cuyos valores son el contador de apariciones de cada emoji representado con variables int, finalmente, de la misma forma que la resolución de q1_memory, leyendo los datos registro a registro para preservar el uso de memoria y ordenando este diccionario para obtener el output.

Se comienza inicializando el diccionario que almacenara los conteos de cada emoji, para luego abrir el archivo usando open() y procesar cada linea de este mediante un for loop, generando una lista de emojis presentes mediante una comprensión de lista que en primer lugar transforma cada linea en un objeto json mediante json.loads() y luego revisa cada caracter del contenido del tweet almacenado en el valor correspondiente a la llave 'content' detectando y agregando emojis a ella solamente si estos están presentes en el diccionario EMOJI_DATA de la librería emoji la cual contiene una lista extensiva de emojis, luego, se puede utilizar otro for loop para iterar sobre los emojis obtenidos y ajustar los contadores respectivos según corresponda. Finalmente, la lista de emojis es ordenada mediante la función sorted obteniendo de esta forma los top 10 emojis más usados con su respectivo conteo, output que es generado tambien por una comprensión de lista que agrega tuplas (emoji, cantidad) por cada elemento en el top 10.

In [4]:
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    #Keep track of emojis
    emojiCount = {}

    #Read file, by line
    with open(file_path, "r") as f:
        for line in f:
            #Parse line, find emojis and add them to the tracker
            emojis = [c for c in json.loads(line)['content'] if c in EMOJI_DATA]
            for emoji in emojis:
                emojiCount[emoji] = emojiCount.get(emoji, 0) + 1

    #Sort emojis by count
    sortedEmojis = sorted(emojiCount, key=lambda e: emojiCount[e], reverse=True)[0:10]
    return [(emoji, emojiCount[emoji]) for emoji in sortedEmojis]

print(
    q2_memory(file_path)
)

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]


#### Time

Al igual que en el problema anterior, la diferencia reside en procesar la información en lotes, por lo que en vez de utilizar contadores, se vuelcan los emojis mismos hacia una lista permitiendo utilizar una variable tipo Counter y su función most_common() para obtener el output deseado.

Luego de inicializar la lista que almacenará los emojis, se lee el archivo completo almacenándolo en una variable de tipo str. A continuación, de la misma manera que en el problema q1_time, se aplica json.loads() a cada elemento del iterable generado al aplicar la función splitlines() sobre el string resultante de la lectura del archivo utilizando la función map(). Luego, al iterar con un for loop sobre el resultado de la función map(), se utiliza el método provisto por la variable EMOJI_DATA de la librería emoji que se indica en el enfoque de memoria del problema Q2 para agregar los emojis encontrados en el valor correspondiente a la llave 'content' de cada registro a la lista de emojis. Finalmente, se genera una variable tipo Counter en base a la lista de emojis sobre la cual se puede utilizar la función most_common(10) obteniendo los top 10 emojis más usados con su respectivo conteo.

In [5]:
def q2_time(file_path: str) -> List[Tuple[str, int]]:
    #Keep track of emojis
    tweetedEmojis = []

    #Read whole file
    with open(file_path, "r") as f:
        rawContent = f.read()

    #Split the file by newlines
    for tweet in map(json.loads, rawContent.splitlines()): #Using splitlines will allow to skip newlines which may be present on the tweet's content
        #Parse tweet, find emojis and add them to the tracker
        tweetedEmojis += [c for c in tweet['content'] if c in EMOJI_DATA]

    return Counter(tweetedEmojis).most_common(10)

print(
    q2_time(file_path)
)

[('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]


### Q3

#### Memory

En este último caso, se requiere encontrar menciones a otros usuarios dentro de cada publicación, lo cual se puede realizar buscando el caracter '@' dentro del contenido de cada una de ellas en consideración que son publicaciones de twitter. Al igual que en los problemas de memoria anteriores se resolvió utilizando contadores, lo cual permite utilizar la función sorted() para ordenarlos y obtener el top 10 requerido. Para detectar el caracter '@' se utilizó la librería re de Python que permite compilar expresiones regulares y así buscar patrones de menciones de usuario dentro del contenido de la publicación.

Comenzando por inicializar el diccionario que almacenará los contadores de cada usuario mencionado, se prosigue con la compilación de la expresión regular '(@\w+)' la cual capturará cualquier combinación alfanumérica incluyendo también el caracter underscore, los cuales justamente comprenden los caracteres permitidos en los handlers de twitter, esta expresión regular permitirá encontrar menciones dentro del contenido de la publicación luego de procesar el archivo como se ha venido haciendo en la resolución de los problemas con enfoque de memoria, es decir registro a registro con un for loop luego de abrirlos con la función open(), luego por cada linea se utiliza la función json.loads() para convertirla en un objeto json y aplicar la función findall() de la expresión regular compilada en el inicio para encontrar todas las menciones dentro del contenido del tweet, específicamente encontrado en el valor de la llave 'content' dentro del objeto json. A continuación, por cada usuario encontrado se aumenta su contador respectivo dentro del diccionario de usuarios mencionados según corresponda. Finalmente, utilizando la función sorted() sobre el diccionario de usuarios mencionados se obtiene el top 10 histórico de usuarios más influyentes, utilizando una comprensión de lista para generar el output requerido, correspondiendo a una lista de tuplas cuyos valores son cada usuario con su cantidad de menciones.

In [6]:
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    #Keep track of mentioned users
    mentionedUsersCount = {}

    #Compile a regex to find '@' mentions
    regex = compile(r'(@\w+)')

    #Read file, by line
    with open(file_path, "r") as f:
        for line in f:
            #Find mentions on tweet
            mentionedUsers = regex.findall(json.loads(line)['content'])

            #Add users to the tracker
            for mentionedUser in mentionedUsers:
                mentionedUsersCount[mentionedUser] = mentionedUsersCount.get(mentionedUser, 0) + 1

    #Sort mentioned users by count then return
    sortedMentionedUsers = sorted(mentionedUsersCount, key=lambda e: mentionedUsersCount[e], reverse=True)[0:10]
    return [(mentionedUser, mentionedUsersCount[mentionedUser]) for mentionedUser in sortedMentionedUsers]

print(
    q3_memory(file_path)
)

[('@narendramodi', 2261), ('@Kisanektamorcha', 1836), ('@RakeshTikaitBKU', 1639), ('@PMOIndia', 1422), ('@RahulGandhi', 1125), ('@GretaThunberg', 1046), ('@RaviSinghKA', 1015), ('@rihanna', 972), ('@UNHumanRights', 962), ('@meenaharris', 925)]


#### Time

Finalmente, se utilizó también la capacidad de las expresiones regulares para encontrar menciones dentro del contenido de cada tweet, diferenciándose del enfoque en memoria respecto a la forma en la cual se lee el archivo y las funciones utilizadas para procesar los datos, los valores obtenidos y realizar el top requerido.

Se inicializa una lista de usuarios mencionados que almacenará cada una de las menciones encontradas en los pasos siguientes, luego, se compila la expresión regular '(@\w+)' la cual cumple la función ya conocida. A continuación de almacenar en una variable str el contenido del archivo, se repite el procesamiento realizado en las respuestas con enfoque de tiempo anteriores, dividiendo la variable str en una lista con la función splitlines() cuyos elementos son procesados por la función json.loads() a través de la función map, resultando en un iterable que pueda ser procesado por un for loop el cual permitirá el uso de la función findall() de la expresión regular compilada en un inicio, con el fin de encontrar las menciones en el contenido del tweet disponible dentro del campo 'content' de cada registro, para así de esta forma agregar cada mención encontrada a la lista de usuarios mencionados. Finalmente, se transforma la lista de usuarios mencionados en una variable tipo Counter, permitiendo utilizar su función most_common(10) para así obtener el top 10 histórico de usuarios más influyentes.

In [7]:
def q3_time(file_path: str) -> List[Tuple[str, int]]:
    #Keep track of mentioned users
    mentionedUsers = []

    #Compile a regex to find '@' mentions
    regex = compile(r'(@\w+)')

    #Read whole file
    with open(file_path, "r") as f:
        rawContent = f.read()

    #Split the file by newlines
    for tweet in map(json.loads, rawContent.splitlines()): #Using splitlines will allow to skip newlines which may be present on the tweet's content
        #Parse tweet, find mentioned users and add them to the tracker
        mentionedUsers += regex.findall(tweet['content'])

    return Counter(mentionedUsers).most_common(10)

print(
    q3_time(file_path)
)

[('@narendramodi', 2261), ('@Kisanektamorcha', 1836), ('@RakeshTikaitBKU', 1639), ('@PMOIndia', 1422), ('@RahulGandhi', 1125), ('@GretaThunberg', 1046), ('@RaviSinghKA', 1015), ('@rihanna', 972), ('@UNHumanRights', 962), ('@meenaharris', 925)]


## Benchmarking

Con el código mencionado en consideración, se utilizaron las librerías memory-profiler y cProfile para realizar los respectivos benchmark de cada una de las respuestas.

* IMPORTANTE: Setear el nombre del ejecutable de Python en el bloque de código siguiente.

In [8]:
# Benchmark setup

pythonExecutable = 'python3'

## Imports

import cProfile
import subprocess
from memory_profiler import profile

### Memory

No hay mucho que indicar respecto al uso de memoria más que destacar que efectivamente se puede apreciar un menor uso de memoria en el enfoque dirigido a este cometido.

In [9]:
# q1_memory

q1MemoryResult = subprocess.run([pythonExecutable, '-m', 'memory_profiler', 'benchmark/q1_memory.py', file_path], stdout=subprocess.PIPE, text=True)
print(q1MemoryResult.stdout)


Filename: benchmark/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7     42.8 MiB     42.8 MiB           1   @profile
     8                                         def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
     9                                             #Keep track of tweet count by date
    10     42.8 MiB      0.0 MiB           1       tweetCountByDate = {}
    11                                         
    12                                             #Keep track of user tweet count by date
    13     42.8 MiB      0.0 MiB           1       tweetUsersByDate = {}
    14                                         
    15                                             #Read file, by line
    16     42.8 MiB      0.0 MiB           1       with open(file_path, "r") as f:
    17     49.4 MiB      0.6 MiB      117408           for line in f:
    18                                                     #Parse line, get username and dat

In [10]:
# q1_time

q1TimeResult = subprocess.run([pythonExecutable, '-m', 'memory_profiler', 'benchmark/q1_time.py', file_path], stdout=subprocess.PIPE, text=True)
print(q1TimeResult.stdout)

Filename: benchmark/q1_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8     42.3 MiB     42.3 MiB           1   @profile
     9                                         def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    10                                             #Open the whole file
    11     42.3 MiB      0.0 MiB           1       with open(file_path, "r") as f:
    12    431.1 MiB    388.8 MiB           1           rawContent = f.read()
    13                                         
    14                                             #Keep track of dates
    15    431.1 MiB      0.0 MiB           1       dates = []
    16                                         
    17                                             #Keep a list for each date, holding the username for each tweet on that date
    18    431.1 MiB      0.0 MiB           1       usersByDate = {}
    19                                         
    20                           

In [11]:
# q2_memory

q2MemoryResult = subprocess.run([pythonExecutable, '-m', 'memory_profiler', 'benchmark/q2_memory.py', file_path], stdout=subprocess.PIPE, text=True)
print(q2MemoryResult.stdout)

Filename: benchmark/q2_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7     52.9 MiB     52.9 MiB           1   @profile
     8                                         def q2_memory(file_path: str) -> List[Tuple[str, int]]:
     9                                             #Keep track of emojis
    10     52.9 MiB      0.0 MiB           1       emojiCount = {}
    11                                         
    12                                             #Read file, by line
    13     52.9 MiB      0.0 MiB           1       with open(file_path, "r") as f:
    14     54.3 MiB      0.9 MiB      117408           for line in f:
    15                                                     #Parse line, find emojis and add them to the tracker
    16     54.3 MiB      0.4 MiB    17386304               emojis = [c for c in json.loads(line)['content'] if c in EMOJI_DATA]
    17     54.3 MiB      0.0 MiB      163043               for emoji in emojis:
    18     54

In [12]:
# q2_time

q2TimeResult = subprocess.run([pythonExecutable, '-m', 'memory_profiler', 'benchmark/q2_time.py', file_path], stdout=subprocess.PIPE, text=True)
print(q2TimeResult.stdout)

Filename: benchmark/q2_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8     53.2 MiB     53.2 MiB           1   @profile
     9                                         def q2_time(file_path: str) -> List[Tuple[str, int]]:
    10                                             #Keep track of emojis
    11     53.2 MiB      0.0 MiB           1       tweetedEmojis = []
    12                                         
    13                                             #Read whole file
    14     53.2 MiB      0.0 MiB           1       with open(file_path, "r") as f:
    15    435.6 MiB    382.5 MiB           1           rawContent = f.read()
    16                                         
    17                                             #Split the file by newlines
    18    863.5 MiB -2858109.1 MiB      117408       for tweet in map(json.loads, rawContent.splitlines()): #Using splitlines will allow to skip newlines which may be present on the tweet's content
    1

In [13]:
# q3_memory

q3MemoryResult = subprocess.run([pythonExecutable, '-m', 'memory_profiler', 'benchmark/q3_memory.py', file_path], stdout=subprocess.PIPE, text=True)
print(q3MemoryResult.stdout)

Filename: benchmark/q3_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7     42.3 MiB     42.3 MiB           1   @profile
     8                                         def q3_memory(file_path: str) -> List[Tuple[str, int]]:
     9                                             #Keep track of mentioned users
    10     42.3 MiB      0.0 MiB           1       mentionedUsersCount = {}
    11                                         
    12                                             #Compile a regex to find '@' mentions
    13     42.3 MiB      0.0 MiB           1       regex = compile(r'(@\w+)')
    14                                         
    15                                             #Read file, by line
    16     42.3 MiB      0.0 MiB           1       with open(file_path, "r") as f:
    17     45.7 MiB      1.0 MiB      117408           for line in f:
    18                                                     #Find mentions on tweet
    19     45.7 M

In [14]:
# q3_time

q3TimeResult = subprocess.run([pythonExecutable, '-m', 'memory_profiler', 'benchmark/q3_time.py', file_path], stdout=subprocess.PIPE, text=True)
print(q3TimeResult.stdout)

Filename: benchmark/q3_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8     42.7 MiB     42.7 MiB           1   @profile
     9                                         def q3_time(file_path: str) -> List[Tuple[str, int]]:
    10                                             #Keep track of mentioned users
    11     42.7 MiB      0.0 MiB           1       mentionedUsers = []
    12                                         
    13                                             #Compile a regex to find '@' mentions
    14     42.7 MiB      0.0 MiB           1       regex = compile(r'(@\w+)')
    15                                         
    16                                             #Read whole file
    17     42.7 MiB      0.0 MiB           1       with open(file_path, "r") as f:
    18    431.4 MiB    388.8 MiB           1           rawContent = f.read()
    19                                         
    20                                             #Split

### Time

Si bien los resultados relativos al tiempo dependen del procesamiento realizado de cara a resolver cada uno de los problemas propuestos, cabe destacar que se puede verificar lo indicado en el apartado Time del Overview de este documento, donde se puede apreciar que la metodología planteada para resolver los problemas con enfoque de tiempo se demora levemente más que la metodología propuesta para resolver los problemas con enfoque en uso de memoria, lo cual podría ser un indicio que un mejor criterio para crear código de mayor calidad sería minimizar el uso de funciones de alto nivel como se realizó en este caso, específicamente map(), splitlines() y most_common(), ya que como se puede apreciar en la metodología del enfoque en memoria, se pueden tener mejores tiempos con un mucho menor uso de recursos. Por otro lado, es importante destacar la disminución de llamadas a funciones que se obtiene con la metodología con enfoque de tiempo que se puede verificar en los benchmark.

In [16]:
# q1_memory
cProfile.run('q1_memory(file_path)')

         1904852 function calls in 2.284 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.509    0.509    2.284    2.284 618638651.py:1(q1_memory)
       13    0.000    0.000    0.000    0.000 618638651.py:23(<lambda>)
    44159    0.002    0.000    0.002    0.000 618638651.py:25(<lambda>)
        1    0.000    0.000    2.284    2.284 <string>:1(<module>)
   117407    0.045    0.000    1.680    0.000 __init__.py:299(loads)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
    49772    0.014    0.000    0.039    0.000 codecs.py:319(decode)
   117407    0.069    0.000    1.619    0.000 decoder.py:332(decode)
   117407    1.497    0.000    1.497    0.000 decoder.py:343(raw_decode)
        1    0.000    0.000    0.001    0.001 interactiveshe

In [17]:
# q1_time
cProfile.run('q1_time(file_path)')

         1643837 function calls in 2.683 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.199    0.199    2.677    2.677 3097206275.py:1(q1_time)
        1    0.007    0.007    2.683    2.683 <string>:1(<module>)
   117407    0.041    0.000    1.666    0.000 __init__.py:299(loads)
       11    0.000    0.000    0.010    0.001 __init__.py:581(__init__)
       11    0.000    0.000    0.001    0.000 __init__.py:600(most_common)
       11    0.000    0.000    0.010    0.001 __init__.py:649(update)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
       11    0.000    0.000    0.000    0.000 abc.py:117(__instancecheck__)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
        1    0.000    0.000    0.068    0.068 codecs.py:319(decode)
   117407    0.065    0.000    1.611    0.000 decoder.

In [18]:
# q2_memory
cProfile.run('q2_memory(file_path)')

         1437310 function calls in 2.665 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.437    0.437    2.665    2.665 2931744933.py:1(q2_memory)
      641    0.000    0.000    0.000    0.000 2931744933.py:14(<lambda>)
        1    0.000    0.000    0.000    0.000 2931744933.py:15(<listcomp>)
   117407    0.515    0.000    0.515    0.000 2931744933.py:9(<listcomp>)
        1    0.000    0.000    2.665    2.665 <string>:1(<module>)
   117407    0.048    0.000    1.670    0.000 __init__.py:299(loads)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
    49772    0.014    0.000    0.039    0.000 codecs.py:319(decode)
   117407    0.067    0.000    1.607    0.000 decoder.py:332(decode)
   117407    1.490    0.000    1.490    0.000 decoder.

In [19]:
# q2_time
cProfile.run('q2_time(file_path)')

         1291513 function calls in 2.971 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.156    0.156    2.966    2.966 2183131300.py:1(q2_time)
   117407    0.499    0.000    0.499    0.000 2183131300.py:12(<listcomp>)
        1    0.005    0.005    2.971    2.971 <string>:1(<module>)
   117407    0.045    0.000    1.624    0.000 __init__.py:299(loads)
        1    0.000    0.000    0.001    0.001 __init__.py:581(__init__)
        1    0.000    0.000    0.000    0.000 __init__.py:600(most_common)
        1    0.000    0.000    0.001    0.001 __init__.py:649(update)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 abc.py:117(__instancecheck__)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
        1    0.000    0.000    0.054    0.054 c

In [20]:
# q3_memory
cProfile.run('q3_memory(file_path)')

         1510777 function calls in 2.197 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.438    0.438    2.197    2.197 717031050.py:1(q3_memory)
    15675    0.001    0.000    0.001    0.000 717031050.py:19(<lambda>)
        1    0.000    0.000    0.000    0.000 717031050.py:20(<listcomp>)
        1    0.000    0.000    2.197    2.197 <string>:1(<module>)
   117407    0.044    0.000    1.663    0.000 __init__.py:299(loads)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
    49772    0.014    0.000    0.038    0.000 codecs.py:319(decode)
   117407    0.066    0.000    1.604    0.000 decoder.py:332(decode)
   117407    1.487    0.000    1.487    0.000 decoder.py:343(raw_decode)
        1    0.000    0.000    0.001    0.001 interactives

In [21]:
# q3_time
cProfile.run('q3_time(file_path)')

         1291529 function calls in 2.617 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.146    0.146    2.612    2.612 2747271002.py:1(q3_time)
        1    0.005    0.005    2.617    2.617 <string>:1(<module>)
   117407    0.040    0.000    1.611    0.000 __init__.py:299(loads)
        1    0.000    0.000    0.006    0.006 __init__.py:581(__init__)
        1    0.000    0.000    0.001    0.001 __init__.py:600(most_common)
        1    0.000    0.000    0.006    0.006 __init__.py:649(update)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 abc.py:117(__instancecheck__)
        1    0.000    0.000    0.000    0.000 codecs.py:260(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
        1    0.000    0.000    0.075    0.075 codecs.py:319(decode)
   117407    0.062    0.000    1.557    0.000 decoder.

## Cloud

La simplicidad y modularidad del código provisto permite un fácil despliegue en la nube, al ser cada función un script simple, se puede desplegar sin mucho esfuerzo en un servicio como GCP Cloud Functions, o bien, puede ser agregado a una imagen Docker con la finalidad de ejecutarlo en algún servicio de despliegue de plataformas Kubernetes, lo cual está disponible en todos los proveedores Cloud con mayor presencia en el mercado, inclusive, se puede utilizar también una imagen Docker para desplegar el código en servicios más específicos como lo es GCP Cloud Run.

## Cierre

Para finalizar, a nivel personal, me pareció una experiencia importante darme cuenta en el proceso de realizar el challenge que había obtenido resultados dispares a lo esperado, y dado este caso, preferí explayarme respecto a estos hallazgos en vez de tratar de llegar a un resultado específico mediante el código, creo que es una experiencia que ameritaba ser escrita y que me permitió volcar mis ideas de una forma que me deja conforme, de la misma forma, espero haber logrado traspasar estas ideas de manera clara tanto en sus descripciones como en el código mismo.

Gracias!